# 집 근처 주유소 10곳 분석

In [1]:
import time
import pandas as pd 
import folium
import requests
from urllib.parse import urlparse
from glob import glob 
from tqdm import tqdm_notebook
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 집 근처인 강서구, 양천구 주유소 데이터만 받기

In [2]:
driver = webdriver.Chrome('./chromedriver')
#driver.maximize_window()
driver.get('http://www.opinet.co.kr/user/main/mainView.do')
time.sleep(1)
driver.find_element_by_css_selector('.ic_m1').click()
time.sleep(2)

region = driver.find_element_by_xpath('//*[@id="SIGUNGU_NM0"]')
gu_list = region.find_elements_by_tag_name('option')

gu_names = [gu.get_attribute('value') for gu in gu_list]
del gu_names[0]

# 강서구, 양천구에 대해서 엑셀 다운로드
for gu in gu_names:
    region = driver.find_element_by_xpath('//*[@id="SIGUNGU_NM0"]')
    region.send_keys(gu)
    time.sleep(1)

    if gu == '강서구' or gu == '양천구':
        driver.find_element_by_xpath('//*[@id="glopopd_excel"]/span').click()
        time.sleep(1)

In [3]:
# 강서, 양천구 주유소에 대해 한 개의 데이터 프레임으로 만든다
station_files = glob('./station_data/집근처*.xls')
tmp_raw = []

for file_name in station_files:
    tmp = pd.read_excel(file_name, header=2)
    tmp_raw.append(tmp)

station_raw = pd.concat(tmp_raw)

In [4]:
station_raw.head()

,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유
0,서울특별시,뉴신정주유소,서울 강서구 곰달래로 207(화곡동),알뜰주유소,02-2601-0077,N,-,1243,1043,-
1,서울특별시,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),알뜰주유소,02-2691-5151,Y,-,1243,1043,-
2,서울특별시,방화주유소,서울 강서구 방화동로 58-2 (방화동),SK에너지,02-2662-0503,N,-,1263,1063,950
3,서울특별시,행촌에너지 공항주유소,서울 강서구 개화동로 420 (개화동),현대오일뱅크,02-2665-5189,Y,-,1264,1064,-
4,서울특별시,강서오곡셀프주유소,서울 강서구 벌말로 254 (오곡동),SK에너지,032-671-5186,Y,-,1265,1065,-


In [5]:
station_list = ['화곡역주유소', '뉴신정주유소', '경인고속주유소', '염창동주유소', '목동주유소', '가로공원주유소', '양천주유소', '건지주유소', '목동주유소', '양천구주유소']
tmp = pd.DataFrame(columns=['지역', '상호', '주소', '상표', '전화번호', '셀프여부', '고급휘발유', '휘발유', '경유', '실내등유'])
for station in station_list:
    result = station_raw.loc[station_raw['상호'].str.contains(station, na=False)]
    tmp = pd.concat([tmp, result])
tmp

,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유
1,서울특별시,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),알뜰주유소,02-2691-5151,Y,-,1243,1043,-
0,서울특별시,뉴신정주유소,서울 강서구 곰달래로 207(화곡동),알뜰주유소,02-2601-0077,N,-,1243,1043,-
6,서울특별시,지에스칼텍스㈜ 경인고속주유소,서울 강서구 국회대로 225 (화곡동),GS칼텍스,02-2608-5151,Y,1416,1266,1072,1100
32,서울특별시,염창동주유소,서울 강서구 양천로 720 (염창동),SK에너지,02-3664-4134,N,-,1688,1468,-
13,서울특별시,목동주유소,서울 양천구 목동중앙로 125 (목동),SK에너지,02-2645-5104,Y,-,1318,1128,-
15,서울특별시,(주)삼표에너지 목동주유소,서울 양천구 목동서로 129 (목동),GS칼텍스,02-2654-5153,N,1518,1318,1128,-
1,서울특별시,가로공원주유소,서울 양천구 가로공원로 165 (신월동),SK에너지,02-2695-3423,N,-,1249,1048,-
14,서울특별시,현대오일뱅크(주) 양천주유소,서울 양천구 안양천로 1179 (목동),현대오일뱅크,02-2653-5189,N,1589,1318,1128,-
11,서울특별시,지에스칼텍스㈜ 건지주유소,서울 양천구 안양천로 1171 (목동),GS칼텍스,02-2644-5108,Y,1495,1308,1118,1030
13,서울특별시,목동주유소,서울 양천구 목동중앙로 125 (목동),SK에너지,02-2645-5104,Y,-,1318,1128,-


In [6]:
# '주소' 를 기준으로 중복되는 행 제거 후 인덱스 초기화
tmp = tmp.drop_duplicates(['주소'])
tmp.reset_index(drop=True, inplace=True)
tmp

,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유
0,서울특별시,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),알뜰주유소,02-2691-5151,Y,-,1243,1043,-
1,서울특별시,뉴신정주유소,서울 강서구 곰달래로 207(화곡동),알뜰주유소,02-2601-0077,N,-,1243,1043,-
2,서울특별시,지에스칼텍스㈜ 경인고속주유소,서울 강서구 국회대로 225 (화곡동),GS칼텍스,02-2608-5151,Y,1416,1266,1072,1100
3,서울특별시,염창동주유소,서울 강서구 양천로 720 (염창동),SK에너지,02-3664-4134,N,-,1688,1468,-
4,서울특별시,목동주유소,서울 양천구 목동중앙로 125 (목동),SK에너지,02-2645-5104,Y,-,1318,1128,-
5,서울특별시,(주)삼표에너지 목동주유소,서울 양천구 목동서로 129 (목동),GS칼텍스,02-2654-5153,N,1518,1318,1128,-
6,서울특별시,가로공원주유소,서울 양천구 가로공원로 165 (신월동),SK에너지,02-2695-3423,N,-,1249,1048,-
7,서울특별시,현대오일뱅크(주) 양천주유소,서울 양천구 안양천로 1179 (목동),현대오일뱅크,02-2653-5189,N,1589,1318,1128,-
8,서울특별시,지에스칼텍스㈜ 건지주유소,서울 양천구 안양천로 1171 (목동),GS칼텍스,02-2644-5108,Y,1495,1308,1118,1030
9,서울특별시,양천구주유소,서울 양천구 국회대로 275 (목동),알뜰주유소,02-2644-5105,Y,-,1259,1062,-


In [8]:
# 필요한 정보만 추출하여 stations 데이터프레임 만들기
stations = pd.DataFrame({
    'Oil_store': tmp['상호'],
    '주소': tmp['주소'],
    '휘발유': tmp['휘발유'],
    '경유': tmp['경유'],
    '셀프': tmp['셀프여부'],
    '상표': tmp['상표']
})
stations

,Oil_store,주소,휘발유,경유,셀프,상표
0,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),1243,1043,Y,알뜰주유소
1,뉴신정주유소,서울 강서구 곰달래로 207(화곡동),1243,1043,N,알뜰주유소
2,지에스칼텍스㈜ 경인고속주유소,서울 강서구 국회대로 225 (화곡동),1266,1072,Y,GS칼텍스
3,염창동주유소,서울 강서구 양천로 720 (염창동),1688,1468,N,SK에너지
4,목동주유소,서울 양천구 목동중앙로 125 (목동),1318,1128,Y,SK에너지
5,(주)삼표에너지 목동주유소,서울 양천구 목동서로 129 (목동),1318,1128,N,GS칼텍스
6,가로공원주유소,서울 양천구 가로공원로 165 (신월동),1249,1048,N,SK에너지
7,현대오일뱅크(주) 양천주유소,서울 양천구 안양천로 1179 (목동),1318,1128,N,현대오일뱅크
8,지에스칼텍스㈜ 건지주유소,서울 양천구 안양천로 1171 (목동),1308,1118,Y,GS칼텍스
9,양천구주유소,서울 양천구 국회대로 275 (목동),1259,1062,Y,알뜰주유소


### '주소' 정보를 통해 위도, 경도 정보 받아오기

In [9]:
key_fd = open('kakaomapkey.txt', mode='r')
kmap_key = key_fd.read(100)
key_fd.close()

In [10]:
lat_list = []; lng_list = []
for i in tqdm_notebook(stations.index):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + stations['주소'][i]
    result = requests.get(urlparse(url).geturl(), headers={"Authorization":"KakaoAK " + kmap_key}).json()
    if result['documents'] == []:
        lat_list.append(np.nan); lng_list.append(np.nan)
    else:
        lat = float(result['documents'][0]['y'])
        lng = float(result['documents'][0]['x'])
        lat_list.append(lat); lng_list.append(lng)

In [11]:
len(stations['Oil_store']), len(stations['주소']), len(stations['휘발유']), len(stations['경유']), len(stations['셀프']), len(stations['상표']), len(lat_list), len(lng_list)

(10, 10, 10, 10, 10, 10, 10, 10)

In [12]:
# 추출한 위도, 경도 정보를 stations 데이터프레임에 추가
stations['위도'] = lat_list
stations['경도'] = lng_list
stations

,Oil_store,주소,휘발유,경유,셀프,상표,위도,경도
0,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),1243,1043,Y,알뜰주유소,37.540768,126.841472
1,뉴신정주유소,서울 강서구 곰달래로 207(화곡동),1243,1043,N,알뜰주유소,37.531562,126.856035
2,지에스칼텍스㈜ 경인고속주유소,서울 강서구 국회대로 225 (화곡동),1266,1072,Y,GS칼텍스,37.529645,126.859835
3,염창동주유소,서울 강서구 양천로 720 (염창동),1688,1468,N,SK에너지,37.549218,126.875677
4,목동주유소,서울 양천구 목동중앙로 125 (목동),1318,1128,Y,SK에너지,37.539601,126.874553
5,(주)삼표에너지 목동주유소,서울 양천구 목동서로 129 (목동),1318,1128,N,GS칼텍스,37.532164,126.875540
6,가로공원주유소,서울 양천구 가로공원로 165 (신월동),1249,1048,N,SK에너지,37.537020,126.834299
7,현대오일뱅크(주) 양천주유소,서울 양천구 안양천로 1179 (목동),1318,1128,N,현대오일뱅크,37.545859,126.881442
8,지에스칼텍스㈜ 건지주유소,서울 양천구 안양천로 1171 (목동),1308,1118,Y,GS칼텍스,37.545316,126.882153
9,양천구주유소,서울 양천구 국회대로 275 (목동),1259,1062,Y,알뜰주유소,37.530373,126.865512


In [35]:
stations['color'] = 'blue'; stations['icon'] = 'tint'

In [36]:
# 휘발유 가격을 기준으로 가장 저렴한 곳 찾은 후 color와 icon 다르게 표시
top1 = stations.sort_values(by='휘발유', ascending=True).head(1)
top1['color'] = 'red'; top1['icon'] = 'star'
top1

,Oil_store,주소,휘발유,경유,셀프,상표,위도,경도,color,icon
0,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),1243,1043,Y,알뜰주유소,37.540768,126.841472,red,star


In [37]:
# 원래 데이터프레임과 병합 후 중복제거(단, top1을 앞에 놓음으로써 제거되지 않게함)
stations = pd.concat([top1, stations])
stations = stations.drop_duplicates(['주소'])
stations.reset_index(drop=True, inplace=True)
stations

,Oil_store,주소,휘발유,경유,셀프,상표,위도,경도,color,icon
0,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),1243,1043,Y,알뜰주유소,37.540768,126.841472,red,star
1,뉴신정주유소,서울 강서구 곰달래로 207(화곡동),1243,1043,N,알뜰주유소,37.531562,126.856035,blue,tint
2,지에스칼텍스㈜ 경인고속주유소,서울 강서구 국회대로 225 (화곡동),1266,1072,Y,GS칼텍스,37.529645,126.859835,blue,tint
3,염창동주유소,서울 강서구 양천로 720 (염창동),1688,1468,N,SK에너지,37.549218,126.875677,blue,tint
4,목동주유소,서울 양천구 목동중앙로 125 (목동),1318,1128,Y,SK에너지,37.539601,126.874553,blue,tint
5,(주)삼표에너지 목동주유소,서울 양천구 목동서로 129 (목동),1318,1128,N,GS칼텍스,37.532164,126.875540,blue,tint
6,가로공원주유소,서울 양천구 가로공원로 165 (신월동),1249,1048,N,SK에너지,37.537020,126.834299,blue,tint
7,현대오일뱅크(주) 양천주유소,서울 양천구 안양천로 1179 (목동),1318,1128,N,현대오일뱅크,37.545859,126.881442,blue,tint
8,지에스칼텍스㈜ 건지주유소,서울 양천구 안양천로 1171 (목동),1308,1118,Y,GS칼텍스,37.545316,126.882153,blue,tint
9,양천구주유소,서울 양천구 국회대로 275 (목동),1259,1062,Y,알뜰주유소,37.530373,126.865512,blue,tint


### 지도 위에 주유소 10곳 표시
- 가장 저렴한 주유소는 따로 표시

In [38]:
map = folium.Map(location=[stations['위도'].mean(), stations['경도'].mean()], zoom_start=14)

for i in stations.index:
    folium.Marker(
        location=[stations['위도'][i], stations['경도'][i]],
        popup=stations['주소'][i],
        tooltip=f"휘발유 가격: {stations['휘발유'][i]}원 / 경유 가격: {stations['경유'][i]}원",
        icon=folium.Icon(color=stations.color[i], icon=stations.icon[i])
    ).add_to(map)
map